In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
import technical_analysis.backtest_kz as bt
from data_pipelines.data_manipulation import DataManipulation
import warnings
warnings.simplefilter(action = 'ignore', category = pd.errors.PerformanceWarning)
%matplotlib inline

In [3]:
SYMBOL = 'SASA.IS'
scale = 1
range = [5, 10, 15, 20, 25, 30]
range = [i*scale for i in range]
period = '1y'
interval = '1d'
start_date = ''
end_data = ''
source = 'yahoo'

In [4]:
data = DataManipulation(SYMBOL, source, range, period=period, interval=interval, scale=scale, prefix_path='..')

In [5]:
df = data.df
sample  = df[['Open','High','Low','Close','Volume']].copy()

,Open,High,Low,Close,Volume,sma_5,sma_10,sma_15,sma_20,sma_25,...,stoch_d,ich_kline,ich_tline,FISHERT_9_1,FISHERTs_9_1,log_rt,candlestick_pattern,candle_label,d_r,feature_label
Datetime,,,,,,,,,,,,,,,,,,,,,
2022-07-19,40.9800,42.1200,40.7800,41.8200,22884660,40.7520,40.6920,41.2947,42.3330,42.7184,...,66.6667,43.2600,40.8600,-0.6890,-1.4142,0.0174,CDLLONGLINE_Bull,1.0000,0.0175,1.0000
2022-07-20,42.0000,43.2200,41.9400,42.7600,23253022,41.0760,40.7880,41.1933,42.1690,42.6704,...,100.0000,43.2600,41.3400,0.0120,-0.6890,0.0222,NO_PATTERN,0.0000,0.0225,1.0000
2022-07-21,42.8600,42.9800,41.8800,42.0000,16565420,41.4680,40.9380,41.0827,41.9930,42.6456,...,66.6667,43.2600,41.3700,0.5831,0.0120,-0.0179,CDLCLOSINGMARUBOZU_Bear,-1.0000,-0.0178,1.0000
2022-07-22,42.2000,42.5400,40.1000,41.3000,12115778,41.7960,41.1040,41.0147,41.7740,42.5152,...,33.3333,43.2600,41.3700,0.6615,0.5831,-0.0168,NO_PATTERN,0.0000,-0.0167,0.0000
2022-07-25,41.3400,42.0800,41.2400,41.9800,14491553,41.9720,41.2720,41.0533,41.6180,42.4056,...,30.2861,43.2600,41.3700,0.6735,0.6615,0.0163,CDLHARAMI_Bull,1.0000,0.0165,0.0000
